## Gerando arquivo para NER-NestedClinBr
Deixar sempre o arquivo de test para fazer o teste completo (com o pipeline todo)
1o. -> passar pelo modelo de NER normal (binario), pegar possiveis entidades
2o. -> combinacao de palavras, modelo de span

Descontinuas: treinar duplicando as frases, tirando os termos do meio.. na hora da predicao: se duas entidades estão mito proximas (ou no mesmo chunk), tirar o espaço e ver se são (ou tentar achar uma regra, ex sempre q tem tratamento e uma anotmia perto, usar uma janela.. ex duas palavras de distancia)...

Para tese: testar tbm sem as descontinuas.. pra pode comparar com outros metodos...

v2 - com corpus original Tempclinbr (arq .txt), contendo as quebras de linha.. aqui quebrar cada frase do texto em uma entrada no dicionario.

v3 - split de train em 75% para dev.. conforme feito no TempClinBr.. e tbm tirar os 41 primeiros tokens dos textos: 
'Data de Criação do Documento: 22/04/2014' como codigo Joao

In [1]:
import os
from pathlib import Path
import re
import pickle
import random


In [2]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

In [3]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

In [4]:
def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        return None

In [410]:
pastasCorpus = [r'corpus\test',r'corpus\train']
#pastasCorpus = [r'corpus\test']
#pastasCorpus = [r'corpus\TESTE']

In [490]:
import re

s = 'Make the World a 2.4Better Place2.0'
pattern = r'([0-9])\.([0-9])'
replacement = r'\1y\2'
html = re.sub(pattern, replacement, s)

print(html)

Make the World a 2y4Better Place2y0


In [496]:
def getDicSentences(pastaCorpus):
    #devePrintar=True
    devePrintar=False
    dic_sentences = {}
    numMaxTokensPorFrase=0 # numero tokens da maior frase
    num=0
    listaEntidades=[]
    frasesComDescontinuas=[]
    for filename in os.listdir(pastaCorpus):
        f = os.path.join(pastaCorpus, filename)
        if os.path.isfile(f):
            fileNameSemExtensao=os.path.splitext(filename)[0]
            if devePrintar:
                print('\n\n--fileName (sem extensao):--', fileNameSemExtensao)
                pass
            extension = os.path.splitext(filename)[1][1:]
            if extension=='ann':
                # tokens da frase
                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+'.txt', "r", encoding='utf-8')
                linha=fileTxt.readlines()
                fileTxt.close()
                if devePrintar:
                    print('linha:', linha)
                    pass
                frases=[]
                allFrasesString=''
                numL=0
                for l in linha:
                    numL=numL+1
                    allFrasesString = allFrasesString+l
                    if numL==1: # descarta primeira frase, que é data de criação do doc
                        #print('descartando frase:', l)
                        continue
                    if l.strip() and l.strip()!='\n':
                        pattern = r'([0-9])\.([0-9])'
                        replacement = r'\1==\2'
                        novoL = re.sub(pattern, replacement, l.strip())
                        l2 = novoL.split('.') # quebrando frases
                        for l3 in l2:
                            if l3.strip() and l3.strip()!='\n':
                                novaFrase = l3.replace('\n','').replace('==','.').strip()+'.'
                                frases.append(novaFrase)
                #print('frases:', frases)
                # para cada frase
                # para tokenizar nesses tokens
                #print('allFrasesString:', allFrasesString)
                frasesTokens={}
                #numCaracteresTotal=42 # primeira frase ignorada
                numIndiceAnterior=0
                for frase in frases:
                    tokens=[]
                    frase2 = frase.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                    frase2 = replaceWhiteSpaces(frase2)
                    frase2 = frase2.split()
                    for numtoken, token in enumerate(frase2):
                        if devePrintar:
                            print('token:', token)
                            print('numIndiceAnterior:', numIndiceAnterior)
                        if token!='.':
                            numCaracteresTotal = allFrasesString.find(token, numIndiceAnterior, len(allFrasesString))
                        else:
                            numCaracteresTotal=numIndiceAnterior+1
                        #tokens.append([token,numtoken])
                        tokens.append([token,numtoken, numCaracteresTotal])
                        numIndiceAnterior = numCaracteresTotal+len(token)-1
                        if numMaxTokensPorFrase<len(tokens):
                            numMaxTokensPorFrase = len(tokens)
                    frasesTokens[frase]=tokens
                    #linhaTokens=linha.copy()    
                if devePrintar:
                    print('frasesTokens:', frasesTokens)
                # agora, as entidades
                fileAnn = open(f, "r", encoding='utf-8')
                linha=fileAnn.readlines()
                fileAnn.close()

                dicEntidades={}
                for entidade_linha in linha:
                    if ';' not in entidade_linha: # tem descontinua?
                        entidade = entidade_linha.split('\t')
                        tipo_entidade = entidade[1]
                        inicio, fim = tipo_entidade.split()[1:3]
                        tipo_entidade = tipo_entidade.split()[0]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio), int(fim))]=[tipo_entidade, replaceWhiteSpaces(termos_entidade)]
                    else:
                        frasesComDescontinuas.append(num)
                        #print('descontinua')
                        entidade = entidade_linha.split('\t')
                        # ex T10	Problema 244 252;279 306	dispneia aos mdoeardos-leves esforço
                        #Problema 244 252;279 306
                        entidade_temp=entidade[1].split(';')
                        entidade1=entidade_temp[0]
                        tipo_entidade = entidade1
                        inicio1, fim1 = tipo_entidade.split()[1:3]
                        tipo_entidade_string = tipo_entidade.split()[0]
                        # mandar só os termos referentes...
                        tamTermo1=int(fim1)-int(inicio1)
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[:tamTermo1]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio1), int(fim1))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]

                        entidade2=entidade_temp[1]
                        inicio2, fim2 = entidade2.split()[0:2]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[tamTermo1:len(termos_entidade)]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio2), int(fim2))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]


                #print('frasesTokens:', frasesTokens)
                indicesDic = sorted(dicEntidades.keys(), key = lambda item: item[0])
                listaIndicesJaUsados = []
                #list_students.sort(key = lambda x: x[1])   #index 1 means second element
                #print(indicesDic)
                #print('dicEntidades:', dicEntidades)
                
                for key, value in frasesTokens.items():
                    for i in indicesDic:
                        tipo_entidade,termos_entidade = dicEntidades[i]
                        ##key (i) = indices old a comparar
                        #print('key:', key)
                        #print('value:', value)
                        #print('i:', i)
                        for token in value:
                            #print('token:', token)
                            if i[0]==token[2]:
                                novo_inicio, novo_fim = [token[1],token[1]+len(termos_entidade.split())]
                                novos_indices=[]
                                for k in range(novo_inicio,novo_fim):
                                    novos_indices.append(k)
                                listaEntidades.append([termos_entidade, novos_indices, tipo_entidade])
                        
                    if len(value)>0:
                        #print('incluindo:', key)
                        dic_sentences[num]=[value, listaEntidades]
                        listaEntidades=[]
                        num=num+1 
        #print(num)
        #if num>318:
        #    break

        #if num>3:
        #    break
    print('numMaxTokensPorFrase:', numMaxTokensPorFrase)
    return dic_sentences, frasesComDescontinuas
            


In [497]:
print('--Treinamento:--', pastasCorpus[1])
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])
print('len(sentences):', len(dic_sentencesTrain))
print('len(Descontinuas):',len(frasesComDescontinuasTrain))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTrain)))

print('--Teste:--', pastasCorpus[0])
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])
print('len(sentences):', len(dic_sentencesTest))
print('len(Descontinuas):',len(frasesComDescontinuasTest))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTest)))


--Treinamento:-- corpus\train
numMaxTokensPorFrase: 192
len(sentences): 1738
len(Descontinuas): 91
len(frasesComDescontinuas): 51
--Teste:-- corpus\test
numMaxTokensPorFrase: 146
len(sentences): 506
len(Descontinuas): 44
len(frasesComDescontinuas): 17


In [498]:
dic_sentencesTest[1]

[[['Em', 0, 59],
  ['acompanhamento', 1, 62],
  ['no', 2, 77],
  ['ambualtorio', 3, 80],
  ['há', 4, 92],
  ['5', 5, 95],
  ['anos', 6, 97],
  ['por', 7, 102],
  ['FA', 8, 106],
  [',', 9, 108],
  ['uso', 10, 110],
  ['de', 11, 114],
  ['marevan', 12, 117],
  ['5mg', 13, 125],
  ['1', 14, 129],
  ['x', 15, 131],
  ['ao', 16, 133],
  ['dia', 17, 136],
  ['.', 18, 139]],
 [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]]

In [499]:
dic_sentencesTrain[0]

[[['Dispneia', 0, 43],
  ['importante', 1, 52],
  ['aos', 2, 63],
  ['esforços', 3, 67],
  ['+', 4, 76],
  ['dor', 5, 78],
  ['tipo', 6, 82],
  ['peso', 7, 87],
  ['no', 8, 92],
  ['peito', 9, 95],
  ['no', 10, 101],
  ['esforço', 11, 104],
  ['.', 12, 111]],
 [['Dispneia importante aos esforços', [0, 1, 2, 3], 'Problema'],
  ['dor tipo peso no peito no esforço', [5, 6, 7, 8, 9, 10, 11], 'Problema'],
  ['peito', [9], 'Anatomia']]]

In [500]:
frasesComDescontinuasTrain[:10]

[3, 15, 30, 30, 30, 30, 55, 55, 65, 65]

In [501]:
# frases com entidades descontinuas
print(len(set(frasesComDescontinuasTrain)))

51


In [502]:
print(dic_sentencesTest[2])

[[['Comorbidades', 0, 142], [':', 1, 154], ['DM', 2, 156], ['há', 3, 159], ['10', 4, 162], ['anos', 5, 165], ['em', 6, 170], ['uso', 7, 173], ['de', 8, 177], ['metformina', 9, 180], ['850mg', 10, 191], ['3', 11, 197], ['cp', 12, 199], ['/', 13, 201], ['dia', 14, 202], [',', 15, 205], ['acarbose', 16, 207], ['1', 17, 216], ['cp', 18, 218], ['/', 19, 220], ['dia', 20, 221], ['e', 21, 225], ['glicazida', 22, 227], ['60mg', 23, 237], ['2', 24, 242], ['cp', 25, 244], ['/', 26, 246], ['dia', 27, 247], ['e', 28, 251], ['insulina', 29, 253], ['(', 30, 262], ['24', 31, 263], ['-', 32, 266], ['0', 33, 268], ['-', 34, 270], ['24', 35, 272], [')', 36, 274], ['.', 37, 275]], [['Comorbidades', [0], 'Problema'], ['DM', [2], 'Problema'], ['metformina 850mg', [9, 10], 'Tratamento'], ['acarbose', [16], 'Tratamento'], ['glicazida 60mg', [22, 23], 'Tratamento'], ['insulina', [29], 'Tratamento']]]


In [503]:

save_obj('dic_sentencesTrainDev',dic_sentencesTrain)
porc=0.76
tamanhoTotal = len(dic_sentencesTrain)
tamanhoTrain = int(tamanhoTotal*porc)
print('tamanhoTrain', tamanhoTrain)
tamanhoDev = tamanhoTotal - tamanhoTrain
print('tamanhoDev', tamanhoDev)
dic_sentencesDev_temp = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[tamanhoTrain:-1]}
dic_sentencesTrain = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[:tamanhoTrain]}
num=0
dic_sentencesDev = {}
for key, value in dic_sentencesDev_temp.items():
    dic_sentencesDev[num] = value
    num=num+1

print('len(dic_sentencesTrain):', len(dic_sentencesTrain))
print('len(dic_sentencesDev):', len(dic_sentencesDev))
print(dic_sentencesTrain[tamanhoTrain-1])
print(dic_sentencesDev[0])

tamanhoTrain 1320
tamanhoDev 418
len(dic_sentencesTrain): 1320
len(dic_sentencesDev): 417
[[['Nega', 0, 119], ['dispneia', 1, 124], [',', 2, 132], ['ortopneia', 3, 134], ['e', 4, 144], ['DNP', 5, 146], ['.', 6, 149]], [['dispneia', [1], 'Problema'], ['ortopneia', [3], 'Problema'], ['DNP', [5], 'Problema']]]
[[['Paciente', 0, 151], ['relata', 1, 160], ['apenas', 2, 167], ['um', 3, 174], ['episodio', 4, 177], ['no', 5, 186], ['momento', 6, 189], ['de', 7, 197], ['gripe', 8, 200], ['.', 9, 205]], [['gripe', [8], 'Problema']]]


In [504]:
dic_sentencesDev[0]

[[['Paciente', 0, 151],
  ['relata', 1, 160],
  ['apenas', 2, 167],
  ['um', 3, 174],
  ['episodio', 4, 177],
  ['no', 5, 186],
  ['momento', 6, 189],
  ['de', 7, 197],
  ['gripe', 8, 200],
  ['.', 9, 205]],
 [['gripe', [8], 'Problema']]]

In [505]:
dic_sentencesDev[1]

[[['HAS', 0, 207],
  [',', 1, 210],
  ['ICC', 2, 212],
  [',', 3, 215],
  ['nega', 4, 217],
  ['DM', 5, 222],
  ['.', 6, 224]],
 [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]

In [506]:
save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesDev',dic_sentencesDev)
save_obj('dic_sentencesTest',dic_sentencesTest)

In [507]:
# gerar arquivo treinamento
f_train = open('nested_train.conll', 'w')

num_entidade_total=0
num_entidade_train=0
num_entidade_dev=0
num_entidade_test=0


for i in range(len(dic_sentencesTrain)):
    tokens = dic_sentencesTrain[i][0]
    ents = dic_sentencesTrain[i][1]
    indiceEnts=[]
    for token in tokens:
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        f_train.write(tokenGravar+' '+tag+'\n')
        num_entidade_train=num_entidade_train+1
    f_train.write('\n')
        
f_train.close()

f_dev = open('nested_dev.conll', 'w')
for i in range(len(dic_sentencesDev)):
    tokens = dic_sentencesDev[i][0]
    ents = dic_sentencesDev[i][1]
    indiceEnts=[]
    for token in tokens:
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        f_dev.write(tokenGravar+' '+tag+'\n')
        num_entidade_dev=num_entidade_dev+1
    f_dev.write('\n')
f_dev.close()


f_test = open('nested_test.conll', 'w')
for i in range(len(dic_sentencesTest)):
    tokens = dic_sentencesTest[i][0]
    ents = dic_sentencesTest[i][1]
    indiceEnts=[]
    for token in tokens:
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        f_test.write(tokenGravar+' '+tag+'\n')
        num_entidade_test=num_entidade_test+1
    f_test.write('\n')
f_test.close()

print('num_entidade_train', num_entidade_train)
print('num_entidade_dev', num_entidade_dev)
print('num_entidade_test:', num_entidade_test)
num_entidade_total=num_entidade_train+num_entidade_dev+num_entidade_test
print('num_entidade_total:', num_entidade_total)

save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesDev',dic_sentencesDev)
save_obj('dic_sentencesTest',dic_sentencesTest)


num_entidade_train 13793
num_entidade_dev 4019
num_entidade_test: 5453
num_entidade_total: 23265


In [509]:
dic_sentencesTrain[0]

[[['Dispneia', 0, 43],
  ['importante', 1, 52],
  ['aos', 2, 63],
  ['esforços', 3, 67],
  ['+', 4, 76],
  ['dor', 5, 78],
  ['tipo', 6, 82],
  ['peso', 7, 87],
  ['no', 8, 92],
  ['peito', 9, 95],
  ['no', 10, 101],
  ['esforço', 11, 104],
  ['.', 12, 111]],
 [['Dispneia importante aos esforços', [0, 1, 2, 3], 'Problema'],
  ['dor tipo peso no peito no esforço', [5, 6, 7, 8, 9, 10, 11], 'Problema'],
  ['peito', [9], 'Anatomia']]]

### Gerar modelos binarios para baseline

In [465]:
def gravarArquivosBinarios(entidade, dic_sentences, tipo):
    # gerar arquivo treinamento
    f_entidade = open('binarios/nested_'+tipo+'_'+entidade+'.conll', 'w')

    num_entidade_total=0
    num_entidade=0

    # TODO - refazer.. qdo vem entidade isolada, nao está gravando...
    print('\nGravando arquivo de {} para entidade {}'.format(tipo, entidade))

    for i in range(len(dic_sentences)):
        tokens = dic_sentences[i][0]
        ents = dic_sentences[i][1]
        indiceEnts=[]
        for token in tokens:
            #print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1]:
                    tag = ent[2]
                    if tag == entidade:
                        num_entidade=num_entidade+1
                    break
            if tag != entidade:
                tag='O'
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            f_entidade.write(tokenGravar+' '+tag+'\n')
            num_entidade_total=num_entidade_total+1
        f_entidade.write('\n')

    f_entidade.close()

    print('num_entidade:', num_entidade)
    print('num_entidade_total:', num_entidade_total)



In [466]:
entidades = ['Problema','Teste','Tratamento','Anatomia']
for entidade in entidades:
    gravarArquivosBinarios(entidade, dic_sentencesTrain, 'train')
    gravarArquivosBinarios(entidade, dic_sentencesDev, 'dev')
    gravarArquivosBinarios(entidade, dic_sentencesTest, 'test')



Gravando arquivo de train para entidade Problema
num_entidade: 2288
num_entidade_total: 13759

Gravando arquivo de dev para entidade Problema
num_entidade: 547
num_entidade_total: 4053

Gravando arquivo de test para entidade Problema
num_entidade: 795
num_entidade_total: 5453

Gravando arquivo de train para entidade Teste
num_entidade: 750
num_entidade_total: 13759

Gravando arquivo de dev para entidade Teste
num_entidade: 176
num_entidade_total: 4053

Gravando arquivo de test para entidade Teste
num_entidade: 308
num_entidade_total: 5453

Gravando arquivo de train para entidade Tratamento
num_entidade: 1187
num_entidade_total: 13759

Gravando arquivo de dev para entidade Tratamento
num_entidade: 376
num_entidade_total: 4053

Gravando arquivo de test para entidade Tratamento
num_entidade: 446
num_entidade_total: 5453

Gravando arquivo de train para entidade Anatomia
num_entidade: 151
num_entidade_total: 13759

Gravando arquivo de dev para entidade Anatomia
num_entidade: 48
num_entidad

## parte 2- Gerar arquivo treinamento para SpanClassification


In [439]:
def areConsecutive(arr):
    # Sort the array
    arr.sort()
    n = len(arr)
    # checking the adjacent elements
    for i in range (1,n):
        if(arr[i]!=arr[i-1]+1):
            return False;
             
    return True;

In [489]:
dic_sentencesTrain

{0: [[['Dispneia', 0, 43],
   ['importante', 1, 52],
   ['aos', 2, 63],
   ['esforços', 3, 67],
   ['+', 4, 76],
   ['dor', 5, 78],
   ['tipo', 6, 82],
   ['peso', 7, 87],
   ['no', 8, 92],
   ['peito', 9, 95],
   ['no', 10, 101],
   ['esforço', 11, 104],
   ['.', 12, 111]],
  [['Dispneia importante aos esforços', [0, 1, 2, 3], 'Problema'],
   ['dor tipo peso no peito no esforço', [5, 6, 7, 8, 9, 10, 11], 'Problema'],
   ['peito', [9], 'Anatomia']]],
 1: [[['Obeso', 0, 113],
   [',', 1, 118],
   ['has', 2, 120],
   [',', 3, 123],
   ['icc', 4, 125],
   ['.', 5, 128]],
  [['Obeso', [0], 'Problema'],
   ['has', [2], 'Problema'],
   ['icc', [4], 'Problema']]],
 2: [[['c', 0, 130],
   ['#', 1, 132],
   ['cintilografia', 2, 134],
   ['miocardica', 3, 148],
   ['para', 4, 159],
   ['avaliar', 5, 164],
   ['angina', 6, 172],
   ['.', 7, 178]],
  [['cintilografia miocardica', [2, 3], 'Teste'],
   ['miocardica', [3], 'Anatomia'],
   ['avaliar', [5], 'Teste'],
   ['angina', [6], 'Problema']]],
 

In [486]:
def getCombinacaoEntidades(dic_predictions):
    num=0
    numDescontinuas=0
    erro_corpus=0
    num_frases_sem_entidade=0
    lista_erro_corpus=list()
    combinacaoEntidadesAll = list()
    combinacaoEntidades = list()
    pulando_termos_postagger = list()
    for key, value in dic_predictions.items():
        num=num+1
        combinacaoEntidades = list()
        #print('key:', key)
        #print(value)
        tokens=value[0].copy()
        print('tokens:', tokens)
        so_tokens = [t[0] for t in tokens]
        entidades=value[1].copy()
        print('so_tokens:', so_tokens)
        print('entidades:', entidades)
        for entidade in entidades:
            erros_entidade = list()
            #print(entidade[1])
            texto_entidade=entidade[0].strip()
            indices = entidade[1]
            tipo_entidade = entidade[2]
            #print('texto_entidade:', texto_entidade)
            frase = so_tokens.copy()
            #print(frase)
            inicio=indices[0]
            fim=indices[-1]
            frase.insert(inicio, '<e1>')
            frase.insert(fim+2, '</e1>')
            if texto_entidade=='-' or texto_entidade=='=' or texto_entidade=='+' or texto_entidade==':' or texto_entidade==',' or texto_entidade=="'" or texto_entidade=='"' or texto_entidade=='.' or texto_entidade==';' or texto_entidade=='/' or texto_entidade=='(' or texto_entidade==')' or texto_entidade=='[' or texto_entidade==']':
                pass
            #print('--texto_entidade--:', texto_entidade)
            #print('frase:', frase)
            #print('frase[inicio+1:fim+2]:', frase[inicio+1:fim+2])
            #print(tokens[indice])
            # o corpus tem alguns problemas, ex tem entidade descontinua sem o CC
            # entao aqui, se não bater, não adicionar no arquivo de treinamento
            # ex frase "The inhibition of c - fos and c - jun expression by IL - 4 in LPS - treated cells was shown to be due to a lower transcription rate of the c - fos and c - jun genes ."
            #print("texto_entidade:", texto_entidade)
            #print("frase[inicio:fim]:", frase[inicio+1:fim+2])
            #print('texto_entidade:', texto_entidade)
            texto_entidade_comparar=texto_entidade.replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = replaceWhiteSpaces(texto_entidade_comparar)
            #print('' '.join(frase[inicio+1:fim+2]).strip():',' '.join(frase[inicio+1:fim+2]).strip())
            texto_frase_comparar = ' '.join(frase[inicio+1:fim+2]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = replaceWhiteSpaces(texto_frase_comparar)
            #print('texto_frase_comparar:', texto_frase_comparar)
            #tem = [palavra.lower() in texto_entidade_comparar for palavra in palavrasDescontinua]
            #tem = [' '+palavra.lower()+' ' in ' '+texto_entidade_comparar+' ' for palavra in palavrasDescontinua]
            #texto_frase_comparar2 = texto_frase_comparar.replace('and not', 'and_not').replace('as well as', 'as_well_as').replace('neither nor', 'neither_nor').replace('but not', 'but_not')
            #print(frase)
            #print('texto_frase_comparar2.split():', texto_frase_comparar2.split())
            #print('texto_frase_comparar2.split():', texto_frase_comparar2.split()[0])
            #tem = [texto_frase_comparar2.split()[-1]==palavra.lower() for palavra in palavrasDescontinua_underline]
            #tem = [' '+palavra.lower()+' ' in ' '+texto_frase_comparar2+' ' for palavra in palavrasDescontinua_underline]
            #tem_boolean = True in tem
            #print([' '.join(frase).strip(), tipo_entidade])
            #if not tem_boolean:
            #if 1==2:
            #    combinacaoEntidades.append([' '.join(frase).strip(), tipo_entidade]) # apendando entidades reais
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            if (texto_entidade_comparar == texto_frase_comparar):
                combinacaoEntidades.append([' '.join(frase).strip(), tipo_entidade]) # apendando entidades reais
            else:
                print('erroooooooooo')
                
                print('key:', key)
                print(value)
        
                print(so_tokens)
                print(entidades)
        
                print('texto_entidade_comparar:', texto_entidade_comparar)
                print('texto_frase_comparar:', texto_frase_comparar)
                print("' '.join(texto_entidade).strip()):", ' '.join(texto_entidade).strip())
                print("' '.join(frase[inicio:fim].strip():", ' '.join(frase[inicio+1:fim+2]).strip())
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                #print('-----indices erro:----', indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])

        for entidade in entidades:
                indices = entidade[1]
                #print('indices:', indices)
                if indices in erros_entidade:
                    continue
                inicio=indices[0]
                fim=indices[-1]
                # agora, fazer a combinacao entre eles.. todas a seguir serão do tipo 'O'           
                for indice in indices:
                    for i in range(indice, fim+1):
                        # ver se nao tem antes
                        frase = so_tokens.copy()
                        termo = frase[indice:i+2]
                        frase.insert(indice, '<e1>')
                        #print(i)

                        frase.insert(i+2, '</e1>')
                        frase_string=' '.join(frase).strip()
                        #print(frase_string)
                        #if frase_string not in combinacaoEntidades:
                        # ver se frase não termina com pontuacao ('.',',',';','-',')','(',']','[','/','"',)
                        devePular = 0
                        if '. </e1>' in frase_string or ', </e1>' in frase_string  or '; </e1>' in frase_string or '- </e1>' in frase_string  or ': </e1>' in frase_string  or '= </e1>' in frase_string  or '/ </e1>' in frase_string  or '( </e1>' in frase_string  or ') </e1>' in frase_string  or '[ </e1>' in frase_string  or '] </e1>' in frase_string  or ': </e1>' in frase_string or 'and </e1>' in frase_string or 'or </e1>' in frase_string:
                            devePular=1
                        if '<e1> .' in frase_string or '<e1> ,' in frase_string  or '<e1> ;' in frase_string or '<e1> -' in frase_string  or '<e1> :' in frase_string  or '<e1> =' in frase_string  or '<e1> /' in frase_string  or '<e1> (' in frase_string  or '<e1> )' in frase_string  or '<e1> [' in frase_string  or '<e1> ]' in frase_string  or '<e1> :' in frase_string  or '<e1> and' in frase_string  or '<e1> or' in frase_string:
                            devePular=1
                        if re.search("<e1> [0-9]* </e1>", frase_string):
                            devePular=1

                        tem_frase = 0
                        for frase in combinacaoEntidades:
                            if frase[0] == frase_string:
                                tem_frase=''
                                break
                        if tem_frase==0 and devePular==0:
                            #print('inserindo:', frase_string)
                            #print('indice:', indice)
                            combinacaoEntidades.append([frase_string, 'O'])
        # shuffle no combinacaoEntidades
        if len(combinacaoEntidades)>0:
            random.shuffle(combinacaoEntidades)
            combinacaoEntidadesAll.append([' '.join(so_tokens).strip(), combinacaoEntidades])
        else:
            num_frases_sem_entidade = num_frases_sem_entidade+1
            combinacaoEntidadesAll.append([])
            #print("key sem entidade", key)
            #print('len(combinacaoEntidades):', len(combinacaoEntidades))
            #print(so_tokens)
        combinacaoEntidades = list()
        if (num % 1000) ==0:
        #if (num %3) == 0:
            print('key:', key)
            #break

    print('numDescontinuas:', numDescontinuas)
    print('erro_corpus:', erro_corpus)
    print('num_frases_sem_entidade:', num_frases_sem_entidade)
    print('len(combinacaoEntidadesAll:)',len(combinacaoEntidadesAll))
    
    return combinacaoEntidadesAll

In [487]:
print('--Train--')
combinacaoEntidadesTrain = getCombinacaoEntidades(dic_sentencesTrain)
#print('\n--Dev--')
#combinacaoEntidadesDev = getCombinacaoEntidades(dic_sentencesDev)
#print('\n--Test--')
#combinacaoEntidadesTest = getCombinacaoEntidades(dic_sentencesTest)

--Train--
tokens: [['Dispneia', 0, 43], ['importante', 1, 52], ['aos', 2, 63], ['esforços', 3, 67], ['+', 4, 76], ['dor', 5, 78], ['tipo', 6, 82], ['peso', 7, 87], ['no', 8, 92], ['peito', 9, 95], ['no', 10, 101], ['esforço', 11, 104], ['.', 12, 111]]
so_tokens: ['Dispneia', 'importante', 'aos', 'esforços', '+', 'dor', 'tipo', 'peso', 'no', 'peito', 'no', 'esforço', '.']
entidades: [['Dispneia importante aos esforços', [0, 1, 2, 3], 'Problema'], ['dor tipo peso no peito no esforço', [5, 6, 7, 8, 9, 10, 11], 'Problema'], ['peito', [9], 'Anatomia']]
tokens: [['Obeso', 0, 113], [',', 1, 118], ['has', 2, 120], [',', 3, 123], ['icc', 4, 125], ['.', 5, 128]]
so_tokens: ['Obeso', ',', 'has', ',', 'icc', '.']
entidades: [['Obeso', [0], 'Problema'], ['has', [2], 'Problema'], ['icc', [4], 'Problema']]
tokens: [['c', 0, 130], ['#', 1, 132], ['cintilografia', 2, 134], ['miocardica', 3, 148], ['para', 4, 159], ['avaliar', 5, 164], ['angina', 6, 172], ['.', 7, 178]]
so_tokens: ['c', '#', 'cintilogra

so_tokens: ['ATRIO', 'ESQUERDO', 'E', 'DIREITO', 'DILATADOS', '.']
entidades: [['ATRIO ESQUERDO E DIREITO DILATADOS', [0, 1, 2, 3, 4], 'Problema'], ['ATRIO ESQUERDO', [0, 1], 'Anatomia'], ['ATRIO', [0], 'Anatomia'], ['DIREITO', [3], 'Anatomia']]
tokens: [['#', 0, 553], ['MEDICAMENTOS', 1, 555], ['EM', 2, 568], ['USO', 3, 571], [':', 4, 574], ['.', 5, 575]]
so_tokens: ['#', 'MEDICAMENTOS', 'EM', 'USO', ':', '.']
entidades: [['MEDICAMENTOS', [1], 'Tratamento']]
tokens: [['WARFARIM', 0, 577], ['5MG', 1, 586], ['.', 2, 589]]
so_tokens: ['WARFARIM', '5MG', '.']
entidades: [['WARFARIM 5MG', [0, 1], 'Tratamento']]
tokens: [['SELOKEN', 0, 590], ['100MG', 1, 598], ['.', 2, 603]]
so_tokens: ['SELOKEN', '100MG', '.']
entidades: [['SELOKEN 100MG', [0, 1], 'Tratamento']]
tokens: [['DIGOXINA', 0, 604], ['0', 1, 613], [',', 2, 614], ['25MG', 3, 615], ['.', 4, 619]]
so_tokens: ['DIGOXINA', '0', ',', '25MG', '.']
entidades: [['DIGOXINA 0 , 25MG', [0, 1, 2, 3], 'Tratamento']]
tokens: [['*', 0, 620], ['N

so_tokens: ['PCR', 'nao', 'reag', '.']
entidades: [['PCR', [0], 'Teste']]
tokens: [['TAP', 0, 1281], ['23', 1, 1285], [',', 2, 1287], ['6', 3, 1288], ['.', 4, 1289]]
so_tokens: ['TAP', '23', ',', '6', '.']
entidades: [['TAP', [0], 'Teste']]
tokens: [['Conduta', 0, 1290], [':', 1, 1297], ['paciente', 2, 1299], ['encaminhado', 3, 1308], ['à', 4, 1320], ['UDT', 5, 1322], ['para', 6, 1326], ['compensação', 7, 1331], ['da', 8, 1343], ['ICC', 9, 1346], ['.', 10, 1349]]
so_tokens: ['Conduta', ':', 'paciente', 'encaminhado', 'à', 'UDT', 'para', 'compensação', 'da', 'ICC', '.']
entidades: [['compensação da ICC', [7, 8, 9], 'Tratamento']]
tokens: [['Retorno', 0, 1351], ['após', 1, 1359], ['internamento', 2, 1364], ['.', 3, 1376]]
so_tokens: ['Retorno', 'após', 'internamento', '.']
entidades: []
tokens: [['Cx', 0, 43], ['bariatrica', 1, 46], ['2x', 2, 57], ['(', 3, 60], ['ultima', 4, 62], ['ha', 5, 69], ['1', 6, 72], [',', 7, 73], ['5a', 8, 74], [')', 9, 77], ['.', 10, 78]]
so_tokens: ['Cx', 'bar

entidades: [['HOLTER', [0], 'Teste']]
tokens: [['OTIMIZO', 0, 282], ['.', 1, 289]]
so_tokens: ['OTIMIZO', '.']
entidades: []
tokens: [['CVD', 0, 290], ['12', 1, 295], [',', 2, 297], ['5', 3, 298], ['12', 4, 300], ['-', 5, 302], ['12', 6, 303], ['DEMAIS', 7, 308], ['MANTIDO', 8, 315], ['.', 9, 322]]
so_tokens: ['CVD', '12', ',', '5', '12', '-', '12', 'DEMAIS', 'MANTIDO', '.']
entidades: [['CVD 12 , 5', [0, 1, 2, 3], 'Tratamento']]
tokens: [['MCP', 0, 43], ['DILATADA', 1, 48], ['.', 2, 56]]
so_tokens: ['MCP', 'DILATADA', '.']
entidades: [['MCP DILATADA', [0, 1], 'Problema']]
tokens: [['CINTILO', 0, 59], ['NEGATIVA', 1, 68], ['E', 2, 78], ['CETS', 3, 80], ['SEM', 4, 86], ['CORONARIOPATIAS', 5, 91], ['.', 6, 106]]
so_tokens: ['CINTILO', 'NEGATIVA', 'E', 'CETS', 'SEM', 'CORONARIOPATIAS', '.']
entidades: [['CINTILO', [0], 'Teste'], ['CETS', [3], 'Teste'], ['CORONARIOPATIAS', [5], 'Problema']]
tokens: [['13', 0, 107], ['-', 1, 109], ['7', 2, 110], ['-', 3, 111], ['15', 4, 112], ['HOLTER', 5, 

entidades: []
tokens: [['BRNF', 0, 542], ['SS', 1, 547], ['.', 2, 549]]
so_tokens: ['BRNF', 'SS', '.']
entidades: [['SS', [1], 'Problema']]
tokens: [['CPP', 0, 550], [':', 1, 553], ['MV', 2, 555], ['+', 3, 557], ['bilateral', 4, 559], [',', 5, 568], ['REDUZIDO', 6, 570], ['EM', 7, 579], ['BASES', 8, 582], ['.', 9, 587]]
so_tokens: ['CPP', ':', 'MV', '+', 'bilateral', ',', 'REDUZIDO', 'EM', 'BASES', '.']
entidades: [['CPP', [0], 'Anatomia'], ['MV + bilateral , REDUZIDO EM BASES', [2, 3, 4, 5, 6, 7, 8], 'Problema']]
tokens: [['ABD', 0, 588], [':', 1, 591], ['globoso', 2, 593], [',', 3, 600], ['flacido', 4, 602], [',', 5, 609], ['sem', 6, 611], ['VCM', 7, 615], ['ou', 8, 619], ['massas', 9, 622], ['palpáveis', 10, 629], ['.', 11, 638]]
so_tokens: ['ABD', ':', 'globoso', ',', 'flacido', ',', 'sem', 'VCM', 'ou', 'massas', 'palpáveis', '.']
entidades: [['ABD', [0], 'Anatomia'], ['VCM', [7], 'Problema'], ['massas palpáveis', [9, 10], 'Problema']]
tokens: [['MMII', 0, 639], [':', 1, 643], ['ed

tokens: [['A', 0, 1179], ['#', 1, 1181], ['ICO', 2, 1183], ['crônica', 3, 1187], ['.', 4, 1194]]
so_tokens: ['A', '#', 'ICO', 'crônica', '.']
entidades: [['ICO crônica', [2, 3], 'Problema']]
tokens: [['P', 0, 1196], ['#', 1, 1198], ['Retorno', 2, 1200], ['em', 3, 1208], ['6', 4, 1211], ['meses', 5, 1213], ['com', 6, 1219], ['eco', 7, 1223], ['+', 8, 1227], ['lab', 9, 1229], ['(', 10, 1234], ['incluindo', 11, 1236], ['TTOG', 12, 1246], ['e', 13, 1251], ['Hb', 14, 1253], ['glicada', 15, 1256], [')', 16, 1264], ['.', 17, 1265]]
so_tokens: ['P', '#', 'Retorno', 'em', '6', 'meses', 'com', 'eco', '+', 'lab', '(', 'incluindo', 'TTOG', 'e', 'Hb', 'glicada', ')', '.']
entidades: [['eco', [7], 'Teste'], ['lab', [9], 'Teste'], ['TTOG', [12], 'Teste'], ['Hb glicada', [14, 15], 'Teste']]
tokens: [['Reforço', 0, 1267], ['MEV', 1, 1275], ['.', 2, 1278]]
so_tokens: ['Reforço', 'MEV', '.']
entidades: [['MEV', [1], 'Tratamento']]
tokens: [['Lucas', 0, 43], [',', 1, 48], ['77', 2, 50], ['ANOS', 3, 53], [

tokens: [['Refere', 0, 213], ['dores', 1, 220], ['por', 2, 226], ['artrose', 3, 230], [',', 4, 237], ['toma', 5, 239], ['dipirona', 6, 244], ['apenas', 7, 253], ['com', 8, 260], ['muita', 9, 264], ['dor', 10, 270], ['.', 11, 273]]
so_tokens: ['Refere', 'dores', 'por', 'artrose', ',', 'toma', 'dipirona', 'apenas', 'com', 'muita', 'dor', '.']
entidades: [['dores por artrose', [1, 2, 3], 'Problema'], ['dipirona', [6], 'Tratamento'], ['muita dor', [9, 10], 'Problema']]
tokens: [['Mãe', 0, 274], ['falecida', 1, 278], ['com', 2, 287], ['problemas', 3, 291], ['cardíacos', 4, 301], ['(', 5, 311], ['?', 6, 312], [')', 7, 313], ['.', 8, 314]]
so_tokens: ['Mãe', 'falecida', 'com', 'problemas', 'cardíacos', '(', '?', ')', '.']
entidades: [['cardíacos', [4], 'Anatomia']]
tokens: [['Pai', 0, 316], [',', 1, 319], ['90anos', 2, 321], [',', 3, 327], ['hipertenso', 4, 329], ['.', 5, 339]]
so_tokens: ['Pai', ',', '90anos', ',', 'hipertenso', '.']
entidades: []
tokens: [['Sem', 0, 341], ['casos', 1, 345],

tokens: [['CHV', 0, 724], ['.', 1, 727]]
so_tokens: ['CHV', '.']
entidades: []
tokens: [['Alimentação', 0, 728], ['saudável', 1, 740], ['.', 2, 748]]
so_tokens: ['Alimentação', 'saudável', '.']
entidades: []
tokens: [['Nega', 0, 749], ['etilismo', 1, 754], ['e', 2, 763], ['tabagismo', 3, 765], ['.', 4, 774]]
so_tokens: ['Nega', 'etilismo', 'e', 'tabagismo', '.']
entidades: [['etilismo', [1], 'Problema'], ['tabagismo', [3], 'Problema']]
tokens: [['Alergia', 0, 776], ['a', 1, 782], ['penicilina', 2, 786], ['.', 3, 796]]
so_tokens: ['Alergia', 'a', 'penicilina', '.']
entidades: []
tokens: [['RS', 0, 797], ['.', 1, 799]]
so_tokens: ['RS', '.']
entidades: []
tokens: [['Obstipação', 0, 800], ['há', 1, 811], ['1', 2, 814], ['mês', 3, 816], [',', 4, 819], ['chegando', 5, 821], ['a', 6, 830], ['ficar', 7, 832], ['5', 8, 838], ['dias', 9, 840], ['sem', 10, 845], ['ir', 11, 849], ['ao', 12, 852], ['banheiro', 13, 855], ['.', 14, 863]]
so_tokens: ['Obstipação', 'há', '1', 'mês', ',', 'chegando', '

tokens: [['PACIENTE', 0, 671], ['SEM', 1, 680], ['QUEIXAS', 2, 684], ['.', 3, 691]]
so_tokens: ['PACIENTE', 'SEM', 'QUEIXAS', '.']
entidades: [['QUEIXAS', [2], 'Problema']]
tokens: [['CATE', 0, 694], ['16', 1, 699], ['/', 2, 701], ['06', 3, 702], ['/', 4, 704], ['15', 5, 705], [':', 6, 707], ['PRÉ', 7, 709], [':', 8, 712], ['RVP', 9, 714], ['2', 10, 718], [',', 11, 719], ['19', 12, 720], ['/', 13, 723], ['/', 14, 723], ['RVS', 15, 726], ['24', 16, 730], ['.', 17, 732]]
so_tokens: ['CATE', '16', '/', '06', '/', '15', ':', 'PRÉ', ':', 'RVP', '2', ',', '19', '/', '/', 'RVS', '24', '.']
entidades: [['CATE', [0], 'Teste']]
tokens: [['PÓS', 0, 740], [':', 1, 743], ['RVP', 2, 745], ['2', 3, 749], [',', 4, 750], ['08', 5, 751], ['/', 6, 754], ['/', 7, 754], ['RVS', 8, 757], ['12', 9, 761], [',', 10, 763], ['59', 11, 764], ['.', 12, 766]]
so_tokens: ['PÓS', ':', 'RVP', '2', ',', '08', '/', '/', 'RVS', '12', ',', '59', '.']
entidades: []
tokens: [['EXAMES', 0, 767], ['LAB', 1, 774], ['21', 2, 77

so_tokens: ['Acompanhamento', 'de', 'cx', 'cardíaca', '.']
entidades: [['cx cardíaca', [2, 3], 'Tratamento'], ['cardíaca', [3], 'Anatomia']]
tokens: [['2', 0, 92], ['irmãos', 1, 94], ['faleceram', 2, 101], ['com', 3, 111], ['infarto', 4, 115], ['fulminante', 5, 123], ['antes', 6, 134], ['dos', 7, 140], ['60', 8, 144], ['anos', 9, 147], ['.', 10, 151]]
so_tokens: ['2', 'irmãos', 'faleceram', 'com', 'infarto', 'fulminante', 'antes', 'dos', '60', 'anos', '.']
entidades: []
tokens: [['Mãe', 0, 153], ['tinha', 1, 157], ['história', 2, 163], ['de', 3, 172], ['angina', 4, 175], [',', 5, 181], ['faleceu', 6, 183], ['com', 7, 191], ['AVC', 8, 195], ['aos', 9, 199], ['70', 10, 203], ['anos', 11, 206], ['.', 12, 210]]
so_tokens: ['Mãe', 'tinha', 'história', 'de', 'angina', ',', 'faleceu', 'com', 'AVC', 'aos', '70', 'anos', '.']
entidades: []
tokens: [['Pai', 0, 212], ['faleceu', 1, 216], ['de', 2, 224], ['IAM', 3, 227], ['com', 4, 231], ['68', 5, 235], ['anos', 6, 238], ['.', 7, 242]]
so_tokens: 

In [445]:
combinacaoEntidadesTest[4]

['DSLP em uso de sinvastatina , marevan 1 cp / dia seg - sab para no alvo sic .',
 [['<e1> DSLP </e1> em uso de sinvastatina , marevan 1 cp / dia seg - sab para no alvo sic .',
   'Problema'],
  ['DSLP em uso de <e1> sinvastatina </e1> , marevan 1 cp / dia seg - sab para no alvo sic .',
   'Tratamento'],
  ['DSLP em uso de sinvastatina , <e1> marevan </e1> 1 cp / dia seg - sab para no alvo sic .',
   'Tratamento']]]

In [397]:
combinacaoEntidadesTrain[1]

['Obeso , has , icc .',
 [['<e1> Obeso </e1> , has , icc .', 'Problema'],
  ['Obeso , has , <e1> icc </e1> .', 'Problema'],
  ['Obeso , <e1> has </e1> , icc .', 'Problema']]]

In [398]:
combinacaoEntidadesDev[0]

['SOLICITO EXAMES LABORATORIAIS E CATETERISMO DIAGNOSTICO PARA VER OSSTENTS IMPLANTADOS .',
 [['SOLICITO EXAMES LABORATORIAIS E CATETERISMO <e1> DIAGNOSTICO </e1> PARA VER OSSTENTS IMPLANTADOS .',
   'O'],
  ['SOLICITO <e1> EXAMES LABORATORIAIS </e1> E CATETERISMO DIAGNOSTICO PARA VER OSSTENTS IMPLANTADOS .',
   'Teste'],
  ['SOLICITO EXAMES <e1> LABORATORIAIS </e1> E CATETERISMO DIAGNOSTICO PARA VER OSSTENTS IMPLANTADOS .',
   'O'],
  ['SOLICITO <e1> EXAMES </e1> LABORATORIAIS E CATETERISMO DIAGNOSTICO PARA VER OSSTENTS IMPLANTADOS .',
   'O'],
  ['SOLICITO EXAMES LABORATORIAIS E <e1> CATETERISMO </e1> DIAGNOSTICO PARA VER OSSTENTS IMPLANTADOS .',
   'O'],
  ['SOLICITO EXAMES LABORATORIAIS E <e1> CATETERISMO DIAGNOSTICO </e1> PARA VER OSSTENTS IMPLANTADOS .',
   'Teste'],
  ['SOLICITO EXAMES LABORATORIAIS E CATETERISMO DIAGNOSTICO PARA VER <e1> OSSTENTS </e1> IMPLANTADOS .',
   'Tratamento']]]

In [91]:
def gravarArquivosTreinamento(combinacaoEntidadesTrain, combinacaoEntidadesDev, combinacaoEntidadesTest):

    # já ir gravando arquivos treinamento, test e dev...
    # pra fazer um teste sem descontinuas
    numTotalEntidades=0
    numTotalEntidadesTrain=0
    numTotalEntidadesDev=0
    numTotalEntidadesTest=0

    f_train = open('span.train', 'w')

    for i, combinacaoEntidades in enumerate(combinacaoEntidadesTrain):
        if len(combinacaoEntidades)>0:
            frase = combinacaoEntidades[0]
            frases_entidade = combinacaoEntidades[1]
            f_train.write(frase+'\n')
            for frase_entidade in frases_entidade:
                f_train.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
                numTotalEntidades=numTotalEntidades+1
                numTotalEntidadesTrain=numTotalEntidadesTrain+1

    f_train.close()

    f_dev = open('span.dev', 'w')

    for i, combinacaoEntidades in enumerate(combinacaoEntidadesDev):
        if len(combinacaoEntidades)>0:
            frase = combinacaoEntidades[0]
            frases_entidade = combinacaoEntidades[1]
            f_dev.write(frase+'\n')
            for frase_entidade in frases_entidade:
                f_dev.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
                numTotalEntidades=numTotalEntidades+1
                numTotalEntidadesDev=numTotalEntidadesDev+1

    f_dev.close()

    f_test = open('span.test', 'w')
    for i, combinacaoEntidades in enumerate(combinacaoEntidadesTest):
        #print(dicSentences[i])
        if len(combinacaoEntidades)>0:
            frase = combinacaoEntidades[0]
            frases_entidade = combinacaoEntidades[1]
            f_test.write(frase+'\n')
            for frase_entidade in frases_entidade:
                f_test.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
                numTotalEntidades=numTotalEntidades+1
                numTotalEntidadesTest=numTotalEntidadesTest+1

    f_test.close()

    print('numTotalEntidades:', numTotalEntidades)
    print('numTotalEntidadesTrain:', numTotalEntidadesTrain)
    print('numTotalEntidadesDev:', numTotalEntidadesDev)
    print('numTotalEntidadesTest:', numTotalEntidadesTest)


In [92]:
gravarArquivosTreinamento(combinacaoEntidadesTrain, combinacaoEntidadesDev, combinacaoEntidadesTest)

numTotalEntidades: 13879
numTotalEntidadesTrain: 8822
numTotalEntidadesDev: 2022
numTotalEntidadesTest: 3035


### parte 2.2 - tentar tirar verbos, CC, etc do O para nao ficar com mto FP
### depois, com nosso pos tagger

In [700]:
# 1o. encontrar os postaggers das entidades...
import xml.dom.minidom

def getDicPosTagger(XMLfileName):
    dicPostagger = {}
    doc = xml.dom.minidom.parse(XMLfileName);
    sentences = doc.getElementsByTagName("sentence")
    for sentence in sentences:
        # ver se tem entidade do interesse
        cons = sentence.getElementsByTagName("cons")
        list_w = sentence.getElementsByTagName("w")
        for i, w in enumerate(list_w):
            texto = w.firstChild.data.lower()
            postagger = w.getAttribute("c")
            if (postagger == '*'):
                postagger = 'NN'
            dicPostagger[texto] = postagger
            for palavra in texto.split('-'):
                if not palavra in dicPostagger.keys():
                    dicPostagger[palavra] = postagger
            for palavra in texto.split('/'):
                if not palavra in dicPostagger.keys():
                    dicPostagger[palavra] = postagger
            if not texto.replace('(','').replace(')','') in dicPostagger.keys():
                dicPostagger[texto] = postagger
    #save_obj('dicPostagger', dicPostagger)
    return dicPostagger

pathCorpus=r'GENIAcorpus3.02.merged_corrigido.xml'
dicPosTagger = getDicPosTagger(pathCorpus)

In [706]:
def tipoPostaggerTokens(entidade_token, dicPostagger):
    postagger = ''
    for p in entidade_token:
        if p.lower() in dicPostagger.keys():
            postagger = postagger + '-' + dicPostagger.get(p.lower())
        elif p.lower().replace(',','') in dicPostagger.keys():
            postagger = postagger + '-' + dicPostagger.get(p.lower().replace(',',''))
        elif p.lower().replace('(','').replace(')','') in dicPostagger.keys():
            postagger = postagger + '-' + dicPostagger.get(p.lower().replace('(','').replace(')',''))
        else:
            for palavra in p.lower().split('/'):
                if palavra in dicPostagger.keys():
                    postagger = postagger + '-' + dicPostagger.get(palavra)
            for palavra in p.lower().split('-'):
                if palavra in dicPostagger.keys():
                    postagger = postagger + '-' + dicPostagger.get(palavra)
    return postagger

tipoPostaggerTokens(['IL','-','2','gene','CD28'], dicPosTagger)

'-NN-NN-CD-NN-NN'

In [732]:
lista_postaggers_entidades = []
for key, value in dicSentences.items():
    entidades = value[1]
    #print(entidades)
    for entidade in entidades:
        pos_tagger=tipoPostaggerTokens(entidade[0], dicPosTagger)
        if pos_tagger=='-JJ':
            print(entidade)
            print(value[0])
        if pos_tagger not in lista_postaggers_entidades:
            lista_postaggers_entidades.append(pos_tagger)
    


[['bHLH'], [10], 'PROTEIN']
[['It', 0], ['contains', 1], ['a', 2], ['basic', 3], ['helix', 4], ['-', 5], ['loop', 6], ['-', 7], ['helix', 8], ['(', 9], ['bHLH', 10], [')', 11], ['motif', 12], ['contiguous', 13], ['with', 14], ['a', 15], ['Per', 16], ['-', 17], ['Arnt', 18], ['-', 19], ['Sim', 20], ['(', 21], ['PAS', 22], [')', 23], ['homology', 24], ['region', 25], ['.', 26]]
[['GSSG'], [16], 'PROTEIN']
[['Because', 0], ['inhibition', 1], ['of', 2], ['DNA', 3], ['-', 4], ['binding', 5], ['activity', 6], ['is', 7], ['associated', 8], ['with', 9], ['increasing', 10], ['intracellular', 11], ['glutathione', 12], ['disulfide', 13], ['levels', 14], ['and', 15], ['GSSG', 16], ['can', 17], ['be', 18], ['shown', 19], ['to', 20], ['inhibit', 21], ['the', 22], ['DNA', 23], ['-', 24], ['binding', 25], ['activity', 26], ['directly', 27], ['in', 28], ['cell', 29], ['-', 30], ['free', 31], ['systems', 32], [',', 33], ['our', 34], ['studies', 35], ['suggest', 36], ['that', 37], ['GSSG', 38], ['is', 39

[['myeloblast'], [20], 'CELL_TYPE']
[['Egr', 0], ['-', 1], ['1', 2], ['antisense', 3], ['oligomers', 4], ['in', 5], ['the', 6], ['culture', 7], ['media', 8], ['blocked', 9], ['macrophage', 10], ['differentiation', 11], ['in', 12], ['both', 13], ['myeloid', 14], ['leukemia', 15], ['cell', 16], ['lines', 17], ['and', 18], ['normal', 19], ['myeloblasts', 20], ['.', 21]]
[['myeloblast'], [16], 'CELL_TYPE']
[['These', 0], ['observations', 1], ['indicate', 2], ['that', 3], ['expression', 4], ['of', 5], ['Egr', 6], ['-', 7], ['1', 8], ['is', 9], ['essential', 10], ['for', 11], ['and', 12], ['restricts', 13], ['differentiation', 14], ['of', 15], ['myeloblasts', 16], ['along', 17], ['the', 18], ['macrophage', 19], ['lineage', 20], ['.', 21]]
[['PRE'], [5], 'DNA']
[['The', 0], ['enhancer', 1], ['activity', 2], ['of', 3], ['the', 4], ['PRE', 5], ['was', 6], ['completely', 7], ['suppressed', 8], ['when', 9], ['the', 10], ['NRE', 11], ['was', 12], ['present', 13], ['.', 14]]
[['hepatic', 'microsome

In [719]:
lista_postaggers_entidades[:10]

['-NN-NN-CD-NN',
 '-NN-NN-NN-NN',
 '-NN',
 '-CD-NN-NN',
 '-NN-NN-NN',
 '-NN-NN-CD',
 '-JJ-NN-NNS',
 '-NN-NN-JJ-NN',
 '-NN-NN',
 '-NN-NN-NN-NN-NN']

In [725]:
print('-CD' in lista_postaggers_entidades)
print('-CC' in lista_postaggers_entidades) # -> só tem 2, um é em maiusculo, excluir
print('-JJ' in lista_postaggers_entidades)
print('-VBN' in lista_postaggers_entidades)
print('-IN' in lista_postaggers_entidades)
print('-TO' in lista_postaggers_entidades)
print('-VBZ' in lista_postaggers_entidades)
print('-NNS' in lista_postaggers_entidades)

True
True
True
False
False
False
False
True


In [833]:
print(len(pulando_termos_postagger))
#print(pulando_termos_postagger)

174085


In [834]:
pulando_termos_postagger[:10]

[[['CD28', 'requires'], '-NN-VBZ'],
 [['5', '-', 'lipoxygenase', '.'], '-CD-NN-NN-.'],
 [['-', 'lipoxygenase', '.'], '-NN-NN-.'],
 [['lipoxygenase', '.'], '-NN-.'],
 [['CD28', 'surface', 'receptor', 'provides'], '-NN-NN-NN-VBZ'],
 [['surface', 'receptor', 'provides'], '-NN-NN-VBZ'],
 [['receptor', 'provides'], '-NN-VBZ'],
 [['interleukin', '-', '2', '('], '-NN-NN-CD-('],
 [['-', '2', '('], '-NN-CD-('],
 [['2', '('], '-CD-(']]

In [835]:
combinacaoEntidadesAll[:1]

[['IL - 2 gene expression and NF - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
  [['IL - <e1> 2 gene </e1> expression and NF - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
    'O'],
   ['IL - 2 gene expression and NF - kappa <e1> B </e1> activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
    'O'],
   ['IL - 2 <e1> gene </e1> expression and NF - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
    'O'],
   ['IL - 2 gene expression and NF - kappa B activation through <e1> CD28 </e1> requires reactive oxygen production by 5 - lipoxygenase .',
    'PROTEIN'],
   ['IL - 2 gene expression and <e1> NF </e1> - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
    'O'],
   ['IL - 2 gene expression and NF - <e1> kappa B </e1> activation through CD28 requires reactive oxygen production b

### Agora, tratar descontinuas

In [836]:
combinacaoEntidades_desc = list()
combinacaoEntidadesAll_desc = list()
palavrasDescontinua=['AND NOT', 'AND/OR', 'AS WELL AS', 'AND', 'OR', 'BUT NOT', 'NEITHER NOR', 'THAN']
pulando_termos_postagger=[]
# agora, tratar descontinuas
for key in frasesComDescontinuas:
    entidades = dicSentencesComLabel[key][1].copy()
    for entidade in entidades:
        indices = entidade[1]
        if not areConsecutive(indices): # é descontinua
            tipo_entidade = entidade[2]
            texto_entidade = ' '.join(entidade[0])
            #print(texto_entidade)
            inicio=indices[0]
            fim=indices[-1]
            tokens = dicSentencesComLabel[key][0].copy()
            so_tokens = [t[0] for t in tokens]
            #print('so_tokens:', so_tokens)
            inicio_frase = so_tokens[:inicio]
            fim_frase = so_tokens[fim+1:-1] 
            #print('inicio_frase:', inicio_frase)
            #print('fim_frase:', fim_frase)
            meio_frase = '<e1> '+texto_entidade + ' </e1> '
            frase_final = ' '.join(inicio_frase)+' '+meio_frase+' '.join(fim_frase)+' .'
            frase_final_texto = ' '.join(inicio_frase)+' '+texto_entidade+' '.join(fim_frase)+' .'
            frase_final = replaceWhiteSpaces(frase_final)
            frase_final_texto = replaceWhiteSpaces(frase_final_texto)
            #print('frase_final_texto:', frase_final_texto)
            #print('frase_final:', frase_final)
            combinacaoEntidades_desc.append([frase_final, tipo_entidade])
            if len(combinacaoEntidades_desc)>0:
                combinacaoEntidadesAll_desc.append([frase_final_texto, combinacaoEntidades_desc])
            combinacaoEntidades_desc = list()
            '''
            #agora, as combinações possiveis
            # primeiro, gerar o termo todo
            menorIndice = entidade[1][0]
            maiorIndice = entidade[1][-1]
            novoIndice=[n for n in range(menorIndice, maiorIndice+1)]
            print(menorIndice)
            print(maiorIndice)
            print('novoIndice:', novoIndice)
            novoTexto=[]
            for token in tokens:
                for indice in novoIndice:
                    if token[1] == indice:
                        novoTexto.append(token[0]) # novo termo com o CC
                        
            novoTextoCC = ' '.join(novoTexto).lower()
            novoTextoCC = novoTextoCC.replace('and not', "and_not").replace('as well as', "as_well_as").replace('neither nor', "neither_no")
            #indices_cc = [palavra.lower() in novoTexto.split() for palavra in palavrasDescontinua]
            #indices2 = [txt.lower().split().index(palavra.lower()) for palavra in palavrasDescontinua]

            # segundo, ver onde está o CC (AND, OR, etc)
            indice_cc=0
            len_cc=0
            for cc in palavrasDescontinua:
                try:
                    indice_cc = novoTextoCC.split().index(cc.lower()) # pega o primero indice da primeira palavra CC
                    len_cc = len(cc.split())
                    break
                except:
                    continue
            
            # terceiro, fazer a combinação com as descontinuas:
            # tudo antes do CC + <pula um token> + tokens (a partir daqui, combinar)
            print('novoTexto[0:-2]:', novoTexto[0:len(novoTexto)])
            termos_antes_cc = novoTexto[:indice_cc]
            #termos_apos_cc = novoTexto[len_cc+3:-1] # nao pega o primeiro apos
            termos_apos_cc = novoTexto[len_cc+3:len(novoTexto)] # nao pega o primeiro apos
            #termos_ultimo_cc = novoTexto[-1] # nao pega o primeiro apos
            print('termos_antes_cc:', termos_antes_cc)
            print('termos_apos_cc:', termos_apos_cc)
            #print('termos_ultimo_cc:', termos_ultimo_cc)
            
            # concatena o termo antes, com os termos apos, mais a ultima palavra          
            # a primeira e ultima palavra tbm é um termo
            #nova_entidade = termos_antes_cc
            nova_entidade = []
            #for i, termo_apos in enumerate(termos_apos_cc):
            for i in range(len(termos_apos_cc), 0, -1):
                #nova_entidade = termos_antes_cc
                #for j in range(i, len(termos_apos_cc)):
                #nova_entidade = nova_entidade +[termos_apos_cc[j]]
                nova_entidade = [termos_apos_cc[i-1]]+nova_entidade
                #for j in range(i, 0, -1):
                    #print('j:',j)
                #    print('add ',termos_apos_cc[j-1])
                #    nova_entidade = [termos_apos_cc[j-1]]+nova_entidade
                #nova_entidade = nova_entidade
                #print('nova_entidade:', termos_antes_cc+nova_entidade)
                #nova_entidade=[]
                frase_final = ' '.join(inicio_frase)+' '+' '.join(termos_antes_cc+nova_entidade)+' '.join(fim_frase)
                frase_final_texto = ' '.join(inicio_frase)+' <e1> '+' '.join(termos_antes_cc+nova_entidade)+' </e1>'+' '.join(fim_frase)
                print('frase_final:', frase_final)
                print('frase_final_texto:', frase_final_texto)
                # só salva se nao tem ainda
                # e ver se está no padrão para salvar
                devePular = 0
                if '. </e1>' in frase_final_texto or ', </e1>' in frase_final_texto  or '; </e1>' in frase_final_texto or '- </e1>' in frase_final_texto  or ': </e1>' in frase_final_texto  or '= </e1>' in frase_final_texto  or '/ </e1>' in frase_final_texto  or '( </e1>' in frase_final_texto  or ') </e1>' in frase_final_texto  or '[ </e1>' in frase_final_texto  or '] </e1>' in frase_final_texto  or ': </e1>' in frase_final_texto or 'and </e1>' in frase_final_texto or 'or </e1>' in frase_final_texto:
                    devePular=1
                print('devePular:', devePular)
                if '<e1> .' in frase_final_texto or '<e1> ,' in frase_final_texto  or '<e1> ;' in frase_final_texto or '<e1> -' in frase_final_texto  or '<e1> :' in frase_final_texto  or '<e1> =' in frase_final_texto  or '<e1> /' in frase_final_texto  or '<e1> (' in frase_final_texto  or '<e1> )' in frase_final_texto  or '<e1> [' in frase_final_texto  or '<e1> ]' in frase_final_texto  or '<e1> :' in frase_final_texto  or '<e1> and' in frase_final_texto  or '<e1> or' in frase_final_texto:
                    devePular=1
                print('devePular:', devePular)
                if re.search("<e1> [0-9]* </e1>", frase_final_texto):
                    devePular=1
                print('devePular:', devePular)
                pos_tagger_termo = tipoPostaggerTokens(termo, dicPosTagger)
                if pos_tagger_termo not in lista_postaggers_entidades:
                    pulando_termos_postagger.append([termo, pos_tagger_termo])
                    devePular=1

                tem_frase = 0
                for frase in combinacaoEntidades:
                    if frase[0] == frase_string:
                        tem_frase=''
                        break
                print('tem_frase:', tem_frase)
                print('devePular:', devePular)
                if tem_frase==0 and devePular==0:
                    combinacaoEntidades_desc.append([frase_final, 'O'])
                    combinacaoEntidadesAll_desc.append([frase_final_texto, combinacaoEntidades_desc])
                combinacaoEntidades_desc= list()
            '''
            
    #break
    
combinacaoEntidadesAll_desc
#pulando_termos_postagger

[['This delay is associated with down - regulation of many erythroid cell - specific genes , including alpha - globin, band 3 , band 4.1 , and the erythroid cell - specific histone H5 .',
  [['This delay is associated with down - regulation of many erythroid cell - specific genes , including <e1> alpha - globin </e1> , band 3 , band 4.1 , and the erythroid cell - specific histone H5 .',
    'DNA']]],
 ['Transient transfection experiments show that several elements in the promoter - proximal region of the IL - 2R alpha gene contribute to IL - 1 responsiveness , most importantly an NF - kappa B site conserved in the human gene .',
  [['Transient transfection experiments show that several elements in the promoter - proximal region of the IL - 2R alpha gene contribute to IL - 1 responsiveness , most importantly an NF - kappa B site conserved in the <e1> human gene </e1> .',
    'DNA']]],
 ['Finally , the status of our current knowledge concerning the roles of transcription factors in the c

In [837]:
len(combinacaoEntidadesAll_desc)

1568

In [838]:
len(combinacaoEntidadesAll)

16614

In [840]:
# já ir gravando arquivos treinamento, test e dev...
# pra fazer um teste sem descontinuas
numTreinamento = len(combinacaoEntidadesAll)*0.8
numTreinamento = int(numTreinamento)
numTeste = (len(combinacaoEntidadesAll) - numTreinamento)/2
numTeste = int(numTeste)
numTotalEntidades=0
numTotalEntidadesTrain=0
numTotalEntidadesDev=0
numTotalEntidadesTest=0

print('numTreinamento:', numTreinamento)
print('numTeste:', numTeste)

f_train = open('genia.train', 'w')
f_dev = open('genia.dev', 'w')
f_test = open('genia.test', 'w')
f_dev_entidades = open('genia_entidades.dev', 'w')
f_test_entidades = open('genia_entidades.test', 'w')

for i, combinacaoEntidades in enumerate(combinacaoEntidadesAll):
    #print(dicSentences[i])
    frase = combinacaoEntidades[0]
    frases_entidade = combinacaoEntidades[1]
    if i<=numTreinamento:
        f_train.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_train.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTrain=numTotalEntidadesTrain+1
    elif i <= numTreinamento + numTeste:
        f_dev.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_dev.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            f_dev_entidades.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesDev=numTotalEntidadesDev+1
    else:
        f_test.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_test.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            f_test_entidades.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTest=numTotalEntidadesTest+1

f_train.close()
f_test.close()
f_dev.close()
f_test_entidades.close()
f_dev_entidades.close()

print('numTotalEntidades:', numTotalEntidades)
print('numTotalEntidadesTrain:', numTotalEntidadesTrain)
print('numTotalEntidadesDev:', numTotalEntidadesDev)
print('numTotalEntidadesTest:', numTotalEntidadesTest)


numTreinamento: 13291
numTeste: 1661
numTotalEntidades: 165571
numTotalEntidadesTrain: 131376
numTotalEntidadesDev: 16986
numTotalEntidadesTest: 17209


In [ ]:
'''
combinacaoEntidades_desc = list()
combinacaoEntidadesAll_desc = list()
palavrasDescontinua=['AND NOT', 'AND/OR', 'AS WELL AS', 'AND', 'OR', 'BUT NOT', 'NEITHER NOR', 'THAN']

# agora, tratar descontinuas
for key in frasesComDescontinuas:
    entidades = dicSentencesComLabel[key][1].copy()
    for entidade in entidades:
        indices = entidade[1]
        if not areConsecutive(indices): # é descontinua
            tipo_entidade = entidade[2]
            inicio=indices[0]
            fim=indices[-1]
            tokens = dicSentencesComLabel[key][0].copy()
            so_tokens = [t[0] for t in tokens]
            inicio_frase = so_tokens[:inicio]
            fim_frase = so_tokens[fim+2:-1] # TODO - mais 2 ou 3?
            print('inicio_frase:', inicio_frase)
            print('fim_frase:', fim_frase)
            texto_entidade = ' '.join(entidade[0])
            meio_frase = '<e1> '+texto_entidade + ' </e1>'
            frase_final = inicio_frase+meio_frase+fim_frase
            frase_final_texto = inicio_frase+texto_entidade+fim_frase
            print('frase_final_texto:', frase_final_texto)
            print('frase_final:', frase_final)
            combinacaoEntidades_desc.append([frase_final, tipo_entidade])
            combinacaoEntidadesAll_desc.append([frase_final_texto, combinacaoEntidades_desc])
            #agora, as combinações possiveis
            # primeiro, gerar o termo todo
            menorIndice = entidade[1][0]
            maiorIndice = entidade[1][-1]
            novoIndice=[n for n in range(menorIndice, maiorIndice+1)]
            #print(menorIndice)
            #print(maiorIndice)
            #print('novoIndice:', novoIndice)
            novoTexto=[]
            for token in tokens:
                for indice in novoIndice:
                    if token[1] == indice:
                        novoTexto.append(token[0]) # novo termo com o CC
                        
            novoTextoCC = ' '.join(novoTexto).lower()
            novoTextoCC = novoTexto.replace('and not', "and_not").replace('as well as', "as_well_as").replace('neither nor', "neither_no")
            #indices_cc = [palavra.lower() in novoTexto.split() for palavra in palavrasDescontinua]
            #indices2 = [txt.lower().split().index(palavra.lower()) for palavra in palavrasDescontinua]

            # segundo, ver onde está o CC (AND, OR, etc)
            indice_cc=0
            len_cc=0
            for cc in palavrasDescontinua:
                try:
                    indice_cc = novoTextoCC.split().index(cc.lower()) # pega o primero indice da primeira palavra CC
                    len_cc = len(cc.split())
                    break
                except:
                    continue
            
            # terceiro, fazer a combinação com as descontinuas:
            # tudo antes do CC + <pula um token> + tokens (a partir daqui, combinar)
            termos_antes_cc = novoTexto[:indice_cc]
            termos_apos_cc = novoTexto[len_cc+1:-1]# nao pega o primeiro apos
            print('termos_antes_cc:', termos_antes_cc)
            print('termos_apos_cc:', termos_apos_cc)
            for i, termo_apos in enumerate(termos_apos_cc):
                nova_entidade = termos_antes_cc
                for j in range(i, len(termos_apos_cc)):                
                    nova_entidade += nova_entidade +' ' + termos_apos_cc[j]
                    print('nova_entidade:', nova_entidade)
            # só salva se nao tem ainda
            
            # quarto: gerar nova frase com a nova combinação (replace das tags especiaid)

    
'''

In [15]:
'''
#dicSentencesComLabel_tratandoDesc = {}

# soma as descontinuas numa entidade só
# ex: alfa and beta globina -> ENT, ENT, ENT, ENT (o 'AND' vira ENT)
# de 17 a 22
for fraseDescontinua in frasesComDescontinuas:
    entidades = dicSentencesComLabel[fraseDescontinua][1]
    tokens = dicSentencesComLabel[fraseDescontinua][0]
    #print(entidades)
    novasEnt = []
    for entidade in entidades:
        #print('entidade:', entidade)
        menorIndice = entidade[1][0]
        maiorIndice = entidade[1][-1]
        novoIndice=[n for n in range(menorIndice, maiorIndice+1)]
        #print(menorIndice)
        #print(maiorIndice)
        #print('novoIndice:', novoIndice)
        novoTexto=[]
        for token in tokens:
            for indice in novoIndice:
                if token[1] == indice:
                    novoTexto.append(token[0])
        #novoTexto = [token[0] for token in tokens where token[1] in novoIndice]
        #print('novoTexto:', novoTexto)
        novaEnt = [novoTexto, novoIndice]
        novasEnt.append(novaEnt)
    dicSentencesComLabel[fraseDescontinua][1] = novasEnt
'''